# Bank Loan Risk & Profitability Analysis

## Objective
To analyze loan performance data in order to understand default risk, customer behavior,
and profitability patterns using historical Lending Club loan data.

## Dataset
Loan Data 
Source: Kaggle


In [1]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', '{:.2f}'.format)

In [2]:
file_path = "D:/python/Bank-Loan-Risk-Analysis/data/loan.csv"

df = pd.read_csv(
    file_path,
    low_memory=False
)


In [3]:
df.head()

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,url,desc,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,next_pymnt_d,last_credit_pull_d,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_il_6m,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit
0,1077501,1296599,5000,5000,4975.00,36 months,10.65%,162.87,B,B2,NaN,10+ years,RENT,24000.00,Verified,Dec-11,Fully Paid,n,https://lendingclub.com/browse/loanDetail.acti...,Borrower added on 12/22/11 > I need to upgra...,credit_card,Computer,860xx,AZ,27.65,0,Jan-85,1,NaN,NaN,3,0,13648,83.70%,9,f,0.00,0.00,5863.16,5833.84,5000.00,863.16,0.00,0.00,0.00,Jan-15,171.62,NaN,May-16,0.00,NaN,1,INDIVIDUAL,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00,0.00,NaN,NaN,NaN,NaN
1,1077430,1314167,2500,2500,2500.00,60 months,15.27%,59.83,C,C4,Ryder,< 1 year,RENT,30000.00,Source Verified,Dec-11,Charged Off,n,https://lendingclub.com/browse/loanDetail.acti...,Borrower added on 12/22/11 > I plan to use t...,car,bike,309xx,GA,1.00,0,Apr-99,5,NaN,NaN,3,0,1687,9.40%,4,f,0.00,0.00,1008.71,1008.71,456.46,435.17,0.00,117.08,1.11,Apr-13,119.66,NaN,Sep-13,0.00,NaN,1,INDIVIDUAL,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00,0.00,NaN,NaN,NaN,NaN
2,1077175,1313524,2400,2400,2400.00,36 months,15.96%,84.33,C,C5,NaN,10+ years,RENT,12252.00,Not Verified,Dec-11,Fully Paid,n,https://lendingclub.com/browse/loanDetail.acti...,NaN,small_business,real estate business,606xx,IL,8.72,0,Nov-01,2,NaN,NaN,2,0,2956,98.50%,10,f,0.00,0.00,3005.67,3005.67,2400.00,605.67,0.00,0.00,0.00,Jun-14,649.91,NaN,May-16,0.00,NaN,1,INDIVIDUAL,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00,0.00,NaN,NaN,NaN,NaN
3,1076863,1277178,10000,10000,10000.00,36 months,13.49%,339.31,C,C1,AIR RESOURCES BOARD,10+ years,RENT,49200.00,Source Verified,Dec-11,Fully Paid,n,https://lendingclub.com/browse/loanDetail.acti...,Borrower added on 12/21/11 > to pay for prop...,other,personel,917xx,CA,20.00,0,Feb-96,1,35.00,NaN,10,0,5598,21%,37,f,0.00,0.00,12231.89,12231.89,10000.00,2214.92,16.97,0.00,0.00,Jan-15,357.48,NaN,Apr-16,0.00,NaN,1,INDIVIDUAL,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,

In [4]:
df.shape

(39717, 111)

In [5]:
df.columns

Index(['id', 'member_id', 'loan_amnt', 'funded_amnt', 'funded_amnt_inv',
       'term', 'int_rate', 'installment', 'grade', 'sub_grade',
       ...
       'num_tl_90g_dpd_24m', 'num_tl_op_past_12m', 'pct_tl_nvr_dlq',
       'percent_bc_gt_75', 'pub_rec_bankruptcies', 'tax_liens',
       'tot_hi_cred_lim', 'total_bal_ex_mort', 'total_bc_limit',
       'total_il_high_credit_limit'],
      dtype='object', length=111)

In [6]:
df.dtypes

id                              int64
member_id                       int64
loan_amnt                       int64
funded_amnt                     int64
funded_amnt_inv               float64
                               ...   
tax_liens                     float64
tot_hi_cred_lim               float64
total_bal_ex_mort             float64
total_bc_limit                float64
total_il_high_credit_limit    float64
Length: 111, dtype: object

In [7]:
df.isnull().mean().sort_values(ascending=False).head(10)

verification_status_joint   1.00
annual_inc_joint            1.00
mo_sin_old_rev_tl_op        1.00
mo_sin_old_il_acct          1.00
bc_util                     1.00
bc_open_to_buy              1.00
avg_cur_bal                 1.00
acc_open_past_24mths        1.00
inq_last_12m                1.00
total_cu_tl                 1.00
dtype: float64

In [8]:
selected_cols = [
    'loan_amnt',
    'term',
    'int_rate',
    'installment',
    'grade',
    'sub_grade',
    'emp_length',
    'annual_inc',
    'dti',
    'purpose',
    'issue_d',
    'loan_status',
    'total_pymnt'
]

df = df[selected_cols]

I removed post-loan and leakage variables to ensure unbiased analysis.
I am simplifying the problem
I am focusing only on business-relevant data
Memory usage drops significantly

In [9]:
df.shape
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39717 entries, 0 to 39716
Data columns (total 13 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   loan_amnt    39717 non-null  int64  
 1   term         39717 non-null  object 
 2   int_rate     39717 non-null  object 
 3   installment  39717 non-null  float64
 4   grade        39717 non-null  object 
 5   sub_grade    39717 non-null  object 
 6   emp_length   38642 non-null  object 
 7   annual_inc   39717 non-null  float64
 8   dti          39717 non-null  float64
 9   purpose      39717 non-null  object 
 10  issue_d      39717 non-null  object 
 11  loan_status  39717 non-null  object 
 12  total_pymnt  39717 non-null  float64
dtypes: float64(4), int64(1), object(8)
memory usage: 3.9+ MB


## Column Selection Strategy

- Selected borrower, loan, time, and outcome-related columns
- Removed identifiers, leakage variables, and unstructured text fields
- Reduced dataset complexity to focus on risk and profitability analysis


In [10]:
df.head()

,loan_amnt,term,int_rate,installment,grade,sub_grade,emp_length,annual_inc,dti,purpose,issue_d,loan_status,total_pymnt
0,5000,36 months,10.65%,162.87,B,B2,10+ years,24000.00,27.65,credit_card,Dec-11,Fully Paid,5863.16
1,2500,60 months,15.27%,59.83,C,C4,< 1 year,30000.00,1.00,car,Dec-11,Charged Off,1008.71
2,2400,36 months,15.96%,84.33,C,C5,10+ years,12252.00,8.72,small_business,Dec-11,Fully Paid,3005.67
3,10000,36 months,13.49%,339.31,C,C1,10+ years,49200.00,20.00,other,Dec-11,Fully Paid,12231.89
4,3000,60 months,12.69%,67.79,B,B5,1 year,80000.00,17.94,other,Dec-11,Current,3513.33


# Data Cleaning

In [11]:
df.dtypes

loan_amnt        int64
term            object
int_rate        object
installment    float64
grade           object
sub_grade       object
emp_length      object
annual_inc     float64
dti            float64
purpose         object
issue_d         object
loan_status     object
total_pymnt    float64
dtype: object

In [12]:
df['int_rate'] = df['int_rate'].str.replace('%', '', regex=False)
df['int_rate'] = df['int_rate'].astype(float)


### What am I doing:  

Removing % symbol  
Converting text → number  
Making column usable for:  
averages  
buckets  
risk analysis  

In [13]:
df['term'] = df['term'].astype(str)
df['term'] = df['term'].str.extract('(\d+)').astype(int)

#Extracts numeric part only (36, 60)
#Converts to integer

In [14]:
# Convert issue_d to Date

df['issue_d'] = pd.to_datetime(df['issue_d'], format='%b-%y')


### Standardize emp_length (Employment Length)

In [15]:
df['emp_length'].unique()

array(['10+ years', '< 1 year', '1 year', '3 years', '8 years', '9 years',
       '4 years', '5 years', '6 years', '2 years', '7 years', nan],
      dtype=object)

In [16]:
df['emp_length'] = df['emp_length'].replace({
    '10+ years': '10',
    '< 1 year': '0',
    'nan': None
})
df['emp_length'] = df['emp_length'].astype(str)
df['emp_length'] = df['emp_length'].str.extract('(\d+)')
df['emp_length'] = df['emp_length'].astype(float)


#### What I achieved:  

Employment length becomes numeric  
Missing values are preserved logically  
Ready for risk analysis  

In [17]:
df['loan_status'].unique()

array(['Fully Paid', 'Charged Off', 'Current'], dtype=object)

In [18]:
default_status = ['Charged Off']
df['default_flag'] = df['loan_status'].isin(default_status).astype(int)


#### Output meaning  

1 → Default (Charged Off)  
0 → Non-default (Fully Paid, Current)  

In [19]:
df.head()

,loan_amnt,term,int_rate,installment,grade,sub_grade,emp_length,annual_inc,dti,purpose,issue_d,loan_status,total_pymnt,default_flag
0,5000,36,10.65,162.87,B,B2,10.00,24000.00,27.65,credit_card,2011-12-01,Fully Paid,5863.16,0
1,2500,60,15.27,59.83,C,C4,0.00,30000.00,1.00,car,2011-12-01,Charged Off,1008.71,1
2,2400,36,15.96,84.33,C,C5,10.00,12252.00,8.72,small_business,2011-12-01,Fully Paid,3005.67,0
3,10000,36,13.49,339.31,C,C1,10.00,49200.00,20.00,other,2011-12-01,Fully Paid,12231.89,0
4,3000,60,12.69,67.79,B,B5,1.00,80000.00,17.94,other,2011-12-01,Current,3513.33,0


In [20]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39717 entries, 0 to 39716
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   loan_amnt     39717 non-null  int64         
 1   term          39717 non-null  int32         
 2   int_rate      39717 non-null  float64       
 3   installment   39717 non-null  float64       
 4   grade         39717 non-null  object        
 5   sub_grade     39717 non-null  object        
 6   emp_length    38642 non-null  float64       
 7   annual_inc    39717 non-null  float64       
 8   dti           39717 non-null  float64       
 9   purpose       39717 non-null  object        
 10  issue_d       39717 non-null  datetime64[ns]
 11  loan_status   39717 non-null  object        
 12  total_pymnt   39717 non-null  float64       
 13  default_flag  39717 non-null  int32         
dtypes: datetime64[ns](1), float64(6), int32(2), int64(1), object(4)
memory usage: 3.9+ MB


### Data Cleaning Summary  

- Converted interest rate and term into numeric form  at
- Standardized employment length into numeric ye  ars
- Created default_flag for risk anal  ysis
- Converted issue date to datetime for time-based ana  lalysis


# Feature Engineering

In [21]:
df['Profit'] = df['total_pymnt'] - df['loan_amnt']

In [24]:
# Income Buckets (Customer Segmentation)

df['income_bucket'] = pd.cut(
    df['annual_inc'],
    bins=[0, 40000, 80000, 120000, 200000, 1000000],
    labels=['Low', 'Mid', 'Upper-Mid', 'High', 'Very High']
)


In [28]:
# Interest Rate Buckets (Risk Segmentation)

df['int_rate_bucket'] = pd.cut(
    df['int_rate'],
    bins=[0, 8, 12, 16, 20, 30],
    labels=['Very Low', 'Low', 'Medium', 'High', 'Very High']
)


In [29]:
# Loan Issue Year

df['issue_year'] = df['issue_d'].dt.year


In [30]:
# Risk Category
df['risk_category'] = np.where(
    (df['default_flag'] == 1) & (df['int_rate'] > 15),
    'High Risk',
    'Low / Moderate Risk'
)


High interest + default → high risk  
Low interest + no default → low risk  

In [39]:
df[['Profit', 'income_bucket', 'int_rate_bucket', 'issue_year', 'risk_category']].head()


,Profit,income_bucket,int_rate_bucket,issue_year,risk_category
0,863.16,Low,Low,2011,Low / Moderate Risk
1,-1491.29,Low,Medium,2011,High Risk
2,605.67,Low,Medium,2011,Low / Moderate Risk
3,2231.89,Mid,Medium,2011,Low / Moderate Risk
4,513.33,Mid,Medium,2011,Low / Moderate Risk


### Feature Engineering Summary

- Created profit metric to analyze loan profitability
- Segmented customers using income buckets
- Bucketed interest rates to analyze risk pricing
- Extracted issue year for time-based analysis
- Created business-friendly risk categories


In [43]:
df.to_csv("D:/python/Bank-Loan-Risk-AnalysisNotebook/loan_cleaned.csv", index=False)
